## Converse API 사용하기

다음 예제는 Converse API를 통해 Nova와 상호작용하는 방법을 보여줍니다 ([API 문서](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html))

In [1]:
import boto3
import json
import base64

# Create a Bedrock Runtime client
client = boto3.client("bedrock-runtime", region_name="us-east-1")

PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0"


### **단일 턴**

In [2]:

# Define your system prompt(s).
system = [
    {
        "text": "You are an experienced publisher. For each user topic, respond with 3 potential book titles in Korean"
    }
]

# Your user prompt
messages = [
    {"role": "user", "content": [{"text": "a child graduating from high school"}]},
]

# Configure the inference parameters.
inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse(
    modelId=LITE_MODEL_ID, messages=messages, system=system, inferenceConfig=inf_params
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print("\n[Response Content Text]")
print(model_response["output"]["message"]["content"][0]["text"])


[Full Response]
{
  "ResponseMetadata": {
    "RequestId": "53ea9416-b6fc-493d-b965-6437d31b0a15",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 06 Apr 2025 14:37:14 GMT",
      "content-type": "application/json",
      "content-length": "374",
      "connection": "keep-alive",
      "x-amzn-requestid": "53ea9416-b6fc-493d-b965-6437d31b0a15"
    },
    "RetryAttempts": 0
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "text": "1. \uace0\ub4f1\ud559\uad50 \uc878\uc5c5\uc744 \ub9de\uc774\ud558\uba70 (As We Celebrate High School Graduation)\n2. \uc0c8 \ubb38\ud131\uc744 \ub118\uc5b4 (Crossing the New Threshold)\n3. \uc878\uc5c5, \uc0c8\ub85c\uc6b4 \uc2dc\uc791 (Graduation, A New Beginning)"
        }
      ]
    }
  },
  "stopReason": "end_turn",
  "usage": {
    "inputTokens": 26,
    "outputTokens": 78,
    "totalTokens": 104
  },
  "metrics": {
    "latencyMs": 588
  }
}

[Response Content Text]
1. 고등학교 졸업을 맞이

### **멀티 턴**

In [3]:

# Define one or more messages using the "user" and "assistant" roles.
messages = [
    {"role": "user", "content": [{"text": "일주일은 몇 일인가요?"}]},
    {"role": "assistant", "content": [{"text": "일주일은 총 7일 입니다."}]},
    {"role": "user", "content": [{"text": "무슨 요일이 가장 첫번째인가요?"}]},
]

# Configure the inference parameters.
inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse(
    modelId=LITE_MODEL_ID, messages=messages, inferenceConfig=inf_params
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print(model_response["output"]["message"]["content"][0]["text"])


[Full Response]
{
  "ResponseMetadata": {
    "RequestId": "8a3c2d1f-d5cd-4705-9005-1ba02d020791",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 06 Apr 2025 14:38:40 GMT",
      "content-type": "application/json",
      "content-length": "812",
      "connection": "keep-alive",
      "x-amzn-requestid": "8a3c2d1f-d5cd-4705-9005-1ba02d020791"
    },
    "RetryAttempts": 0
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "text": "\ub300\ud55c\ubbfc\uad6d\uc5d0\uc11c\ub294 \uc77c\uc694\uc77c\uc774 \uac00\uc7a5 \uccab \ubc88\uc9f8 \uc694\uc77c\uc785\ub2c8\ub2e4. \ub300\ud55c\ubbfc\uad6d \uad6d\ub0b4 \uae30\uc900\uc73c\ub85c \uc694\uc77c\uc740 \uc77c\uc694\uc77c, \uc6d4\uc694\uc77c, \ud654\uc694\uc77c, \uc218\uc694\uc77c, \ubaa9\uc694\uc77c, \uae08\uc694\uc77c, \ud1a0\uc694\uc77c \uc21c\uc11c\uc785\ub2c8\ub2e4. \ud558\uc9c0\ub9cc \ub2e4\ub978 \ub098\ub77c\uc5d0 \ub530\ub77c \uccab \ubc88\uc9f8 \uc694\uc77c\uc774 \ub

### **스트리밍**

In [4]:

# Define your system prompt(s).
system = [
    {
        "text": "Act as a creative writing assistant. When the user provides you with a topic, write a 20~30 words Korean story about that topic."
    }
]

messages = [
    {"role": "user", "content": [{"text": "캠핑"}]},
]

# Configure the inference parameters.
inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse_stream(
    modelId=LITE_MODEL_ID, messages=messages, system=system, inferenceConfig=inf_params
)

stream = model_response.get("stream")
if stream:
    for event in stream:
        if "contentBlockDelta" in event:
            print(event["contentBlockDelta"]["delta"]["text"], end="")

별빛 아래 캠핑을 하던 그날, 친구들과 함께 불빛을 따뜻하게 받으며 이야기를 나눴다. 불꽃놀이가 시작되었고, 하늘은 숨막힐 듯 아름다웠다. 그 순간, 우리는 행복했다.

### **이미지 이해**

In [8]:

with open("media/sunset.png", "rb") as f:
    image = f.read()

messages = [
    {
        "role": "user",
        "content": [
            {"image": {"format": "png", "source": {"bytes": image}}},
            {"text": "Describe the following image."},
        ],
    }
]

inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse(
    modelId=LITE_MODEL_ID, messages=messages, inferenceConfig=inf_params
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print(model_response["output"]["message"]["content"][0]["text"])


[Full Response]
{
  "ResponseMetadata": {
    "RequestId": "d418ca22-ac4c-4fea-a149-5a43efe99ba9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 06 Apr 2025 14:41:26 GMT",
      "content-type": "application/json",
      "content-length": "797",
      "connection": "keep-alive",
      "x-amzn-requestid": "d418ca22-ac4c-4fea-a149-5a43efe99ba9"
    },
    "RetryAttempts": 0
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "text": "The image depicts a serene sunset scene. The sun is setting on the horizon, casting a warm, golden light across the sky. The sky is a blend of vibrant orange and yellow hues, transitioning into deeper shades of blue as it moves towards the top of the image. The sun is partially obscured by a silhouette of mountains, adding depth to the landscape. The water below the horizon reflects the colors of the sky, creating a mirror-like effect. A few birds are flying in the sky, adding a touch of

In [9]:

# Open the image you'd like to use and encode it as a Base64 string.
with open("media/dog.jpeg", "rb") as image_file:
    dog_binary_data = image_file.read()

with open("media/cat.jpeg", "rb") as image_file:
    cat_binary_data = image_file.read()

messages = [
    {
        "role": "user",
        "content": [
            {"image": {"format": "jpeg", "source": {"bytes": dog_binary_data}}},
            {"image": {"format": "jpeg", "source": {"bytes": cat_binary_data}}},
            {"text": "What do these two images have in common"},
        ],
    }
]

inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse(
    modelId=LITE_MODEL_ID, messages=messages, inferenceConfig=inf_params
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print(model_response["output"]["message"]["content"][0]["text"])


[Full Response]
{
  "ResponseMetadata": {
    "RequestId": "b502274b-9441-44d9-96b6-fa21be0aae97",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 06 Apr 2025 14:41:37 GMT",
      "content-type": "application/json",
      "content-length": "665",
      "connection": "keep-alive",
      "x-amzn-requestid": "b502274b-9441-44d9-96b6-fa21be0aae97"
    },
    "RetryAttempts": 0
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "text": "The two images have several things in common. Firstly, both images feature animals, specifically a dog and a cat. Secondly, both animals are looking directly at the camera, with the dog appearing to be smiling and the cat looking curious. Thirdly, both images have a white background, which helps to highlight the animals and make them stand out. Finally, both images have a close-up view of the animals, which allows the viewer to see the details of their faces and expressions."
        }
 

### **비디오 이해**

In [10]:

with open("./media/the-sea.mp4", "rb") as file:
    media_bytes = file.read()
    media_base64 = base64.b64encode(media_bytes)


messages = [
    {
        "role": "user",
        "content": [
            {"video": {"format": "mp4", "source": {"bytes": media_bytes}}},
            {"text": "Describe the following video"},
        ],
    }
]

inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse(
    modelId=LITE_MODEL_ID, messages=messages, inferenceConfig=inf_params
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print("\n[Response Content Text]")
print(model_response["output"]["message"]["content"][0]["text"])


[Full Response]
{
  "ResponseMetadata": {
    "RequestId": "326b0b4c-b6c7-4aff-9783-64d9c31b25b8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 06 Apr 2025 14:41:44 GMT",
      "content-type": "application/json",
      "content-length": "532",
      "connection": "keep-alive",
      "x-amzn-requestid": "326b0b4c-b6c7-4aff-9783-64d9c31b25b8"
    },
    "RetryAttempts": 0
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "text": "The video starts with a top view of a rocky shore with waves crashing on the rocks. The camera then zooms in on a large seashell lying on the sand. The shell is brown and white in color, with a spiral pattern. The waves are gently washing over the shell, and the sand is wet. The sun is shining brightly, and the light is reflecting on the water."
        }
      ]
    }
  },
  "stopReason": "end_turn",
  "usage": {
    "inputTokens": 3462,
    "outputTokens": 78,
    "totalTokens": 3540
 

### 비디오 이해를 위한 S3 경로

##### 비디오 파일이 위치한 S3 버킷 URI로 대체하세요

In [ ]:

messages = [
    {
        "role": "user",
        "content": [
            {
                "video": {
                    "format": "mp4",
                    "source": {
                        "s3Location": {
                            #Replace the s3 bucket URI 
                            "uri": "s3://demo-bucket/the-sea.mp4"
                        }
                    },
                }
            },
            {"text": "Describe the following video"},
        ],
    }
]

inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse(
    modelId=LITE_MODEL_ID, messages=messages, inferenceConfig=inf_params
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print("\n[Response Content Text]")
print(model_response["output"]["message"]["content"][0]["text"])